# Class Activation Mapping


### Importing the Datasets

In [ ]:
from torchvision import datasets , transforms
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim



transform = transforms.Compose([transforms.ToTensor()])
trainset = datasets.FashionMNIST('MNIST_data/', download = True, train = True, transform = transform)
testset = datasets.FashionMNIST('MNIST_data/', download = True, train = False, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 600, shuffle = True)
testloader = torch.utils.data.DataLoader(testset, batch_size = 1, shuffle = True)

Extracting MNIST_data/FashionMNIST\raw\train-images-idx3-ubyte.gz to MNIST_data/FashionMNIST\raw


Extracting MNIST_data/FashionMNIST\raw\train-labels-idx1-ubyte.gz to MNIST_data/FashionMNIST\raw



Extracting MNIST_data/FashionMNIST\raw\t10k-images-idx3-ubyte.gz to MNIST_data/FashionMNIST\raw


## Viewing samples

In [3]:
dataiter = iter(trainloader)

image,label = dataiter.next()
print(image.shape)

plt.imshow(image[0].numpy().squeeze())

NameError: name 'trainloader' is not defined

## Neural Model for CAM

In [ ]:
class camnet(nn.Module):
    def __init__(self):
        super(camnet,self).__init__()
        
        self.convlayers = nn.Sequential(
            nn.Conv2d(1,6,kernel_size=(3,3)),
            nn.BatchNorm2d(6),
            nn.ReLU(inplace=True), #26x26
            nn.MaxPool2d(kernel_size=(2,2),stride=2), #13x13
            nn.Conv2d(6,16,kernel_size=(2,2)),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),#12x12
            nn.MaxPool2d(kernel_size=(2,2),stride=2),#6x6
            nn.Conv2d(16,120,kernel_size=(2,2)),
            nn.BatchNorm2d(120),
            nn.ReLU(inplace=True),#5x5
            nn.Conv2d(120,240,kernel_size=(2,2)), #4x4
            nn.BatchNorm2d(240))
        
        self.last_layer = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=(4,4))) # pooling average value from each channel

        self.gap_layer = nn.Sequential(
            nn.Linear(240,10),
            nn.Softmax(dim=-1))
    
    
    def forward(self,x):
        output = self.convlayers(x)
        output = self.last_layer(output)
        output = torch.squeeze(output)
        output = self.gap_layer(output)
        
        
        return output
    
cam = camnet()
cam.cuda()
cam_criterion = nn.CrossEntropyLoss()
cam_optimizer = optim.SGD(cam.parameters(),lr = 0.1 , momentum=0.9)


In [ ]:
cam.load_state_dict(torch.load('C:\\Users\\Granite\\Desktop\\camnet.pth'))

loss_per_epoch = []
def train(epoch):
    cam.train()
    
    for i in range(epoch):
        running_loss = 0
        avg_loss = 0
        for j , batch in enumerate(trainloader):
            
            
            image,label = batch
            
            image = image.cuda()
            label = label.cuda()
            
            
            output =cam(image)
            
            cam_optimizer.zero_grad()
            loss = cam_criterion(output,label)
            
            
            
            loss.backward()
            
            cam_optimizer.step()
            
            running_loss += loss.detach()
            
            avg_loss = running_loss/100
        
        print('average loss '+str(avg_loss.item())+' on '+str(i)+' epoch') 
        loss_per_epoch.append(avg_loss.item())


In [ ]:
train(10) #actually i ran it 800 times

In [ ]:
torch.save(cam.state_dict(),'C:\\Users\\Granite\\Desktop\\camnet.pth') #the whole parameter model is saved and given with the assignment

## Upsampling module

In [ ]:
upsample = nn.Upsample(size=(28,28),mode='bilinear') #upsampling method
relu = nn.ReLU(inplace=True)

### function for creating CAM

In [ ]:
def create_cam(feature_map,net,index):
    matrix = net.gap_layer[0].weight
    relued = relu(feature_map)
    relued = torch.squeeze(relued)
    relued_flattened = relued.view(-1,16)
    pulled_vector = matrix[index,:]
    #print(relued_flattened)
    for i in range(240):
        relued_flattened[i,:] = pulled_vector[i]*relued_flattened[i,:]
    reshaped = torch.reshape(relued_flattened,(240,4,4))
    output = upsample(torch.unsqueeze(torch.unsqueeze(torch.sum(reshaped,0),0),0))
    return output

In [ ]:
def test():
    cam.eval()
    correct,total = 0,0
    with torch.no_grad():
        for data in testloader:
            images,labels = data
           
            images = images.cuda()
            
            #print('shape of image', images.shape)
            labels = labels.cuda()
            f_map = cam.convlayers(images)
            l_layer = torch.squeeze(cam.last_layer(f_map))
            print('l_layer shape',l_layer.shape)
            outputs = cam.gap_layer(l_layer)
            print('outputs shape',outputs.shape)
      
            print('feature_map shape',f_map.shape)
            outputs = torch.squeeze(outputs)
        
            _,predicted = torch.max(outputs.data,0)
            total += 1
            corrected = predicted==labels
        
            if corrected:
            
                cam_image = create_cam(f_map,cam,predicted).cpu().numpy()
                cam_image = np.squeeze(cam_image)
                plt.imshow(cam_image)
                plt.pause(0.1)
                plt.imshow(np.squeeze(images.cpu().numpy()))
                plt.pause(0.1)
                print('predicted for ',labels)
                #break
           
      
            
            #print(labels,predicted,corrected)  
            correct += (predicted==labels).sum().item()
        accuracy = (correct/total)*100
    
        #print('correct on test set '+ str(correct))
       # print('accuracy on test set '+ str(accuracy))
    

### Instead of overlapping, class activation mapping is shown followed by the corresponding correctly classified image

In [ ]:
test()

In [ ]:
cam


In [ ]:
cam.gap_layer[0].weight.shape